In [ ]:

def create_degradation_df(df_degradation, df_eval, df_consistency):
    df_eval_for_degradation = df_eval[df_degradation.columns].copy()
    df_consistency_for_degradation = pd.concat([df_consistency]*len(df_eval))
    df_consistency_for_degradation = df_consistency_for_degradation[
        df_degradation.columns].copy()
    df_consistency_for_degradation.index = df_eval_for_degradation.index

    df_consistency_for_degradation['type'] = 'H-H'
    df_eval_for_degradation['type'] = 'H-M'
    df_degradation['type'] = 'diff'

    df = pd.concat(
        [df_consistency_for_degradation,
         df_eval_for_degradation,
         df_degradation])
    return df


df_consistency_old = outputs_old.get('df_consistency', None)
df_degradation_old = outputs_old.get('df_degradation', None)

df_consistency_new = outputs_new.get('df_consistency', None)
df_degradation_new = outputs_new.get('df_degradation', None)

if (df_consistency_old is not None and df_consistency_new is not None
        and df_degradation_new is not None and df_degradation_old is not None):

    markdown_strs = ['## Consistency']

    # human human agreement
    markdown_strs.append('### Human-human agreement')
    display(Markdown('\n'.join(markdown_strs)))
    df_consistency_old['version'] = 'old'
    df_consistency_new['version'] = 'new'
    df = pd.DataFrame(df_consistency_old, copy=True)
    df = df.append(df_consistency_new)
    df.index.name = 'for'
    df = df.reset_index().sort_values(by=['for', 'version'],
                                      ascending=[True, False]).set_index(df.index.names)
    df.index.name = None

    # put version first
    df = df[['version'] + [x for x in df.columns if x != 'version']]
    display(HTML(df.to_html(index=False,
                            escape=False,
                            float_format=float_format_func,
                            classes=['alternate_colors2'])))

    # degradation
    display(Markdown('### Degradation'))
    df_eval_old = outputs_old['df_eval_for_degradation']
    df_eval_new = outputs_new['df_eval_for_degradation']

    # old
    df_degradation_old = create_degradation_df(
        df_degradation_old, df_eval_old, df_consistency_old)
    df_degradation_old['version'] = 'old'

    # new
    df_degradation_new = create_degradation_df(
        df_degradation_new, df_eval_new, df_consistency_new)
    df_degradation_new['version'] = 'new'

    df = pd.concat([df_degradation_old, df_degradation_new])
    df = df[['type', 'version', 'corr', 'kappa',
             'wtkappa', 'exact_agr', 'adj_agr', 'SMD']]

    df.index.name = 'for'
    df = df.reset_index().sort_values(by=['for', 'version', 'type'],
                                      ascending=[True, False, True]).set_index(df.index.names)
    df.index.name = None

    # display the HTML for the table with the various formatters
    display(HTML(df.to_html(float_format=float_format_func,
                            classes=['alternate_colors3_groups'], escape=False)))
